In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("Jupyter") \
    .config("spark.master", "spark://spark-master:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.cores", "2") \
    .config("spark.driver.memory", "2g") \
    .config("spark.cores.max", "4") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .enableHiveSupport()

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark

:: loading settings :: url = jar:file:/usr/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-18a3159c-00c9-4d36-93f1-0f4de7f85f7c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 126ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0  

## Testando Conexão HDFS

In [2]:
#Criando diretorio
!hdfs dfs -mkdir /users/Teste

In [3]:
#movendo arquivos do local para o HDFS
!hdfs dfs -put /usr/notebooks/curva.csv /users/Teste

In [4]:
#Listando diretorios/arquivos
!hdfs dfs -ls /users/Teste/

Found 1 items
-rw-r--r--   1 root supergroup         46 2024-08-18 02:37 /users/Teste/curva.csv


## Testando Hive

In [5]:
spark.sql('CREATE DATABASE IF NOT EXISTS teste')

DataFrame[]

In [6]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|    teste|
+---------+



## Testando Spark

In [7]:
spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [8]:
#Lendo arquivos do HDFS
df = spark.read.csv('/users/Teste/curva.csv',header=True,sep=';')

df.show()

+-----+-------+
|curva|funding|
+-----+-------+
|SELIC|   0.75|
|SELIC|   0.65|
|SELIC|   0.70|
+-----+-------+



In [9]:
#salvando tabela delta
df.write.format("delta").mode("overwrite").saveAsTable('teste.curva')

24/08/18 02:37:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/08/18 02:37:59 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`teste`.`curva` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


In [10]:
#Verificando resultados
spark.sql("select * from teste.curva").show()

+-----+-------+
|curva|funding|
+-----+-------+
|SELIC|   0.75|
|SELIC|   0.65|
|SELIC|   0.70|
+-----+-------+



## Testando SQL Server

In [11]:
# jdbc_url = "jdbc:sqlserver://sqlserver:1433;databaseName=testeDani"
# connection_properties = {
#     "user": "SA",
#     "password": "YourStrong!Passw0rd",
#     "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
# }

# df = spark.read.jdbc(url=jdbc_url, table="daniBoy", properties=connection_properties)
# df.show()


+-----+---+
| Nome| ID|
+-----+---+
| Dani|  1|
| Jess|  2|
|Jamal|  3|
+-----+---+

